In [1]:

import os
import json
from google.cloud import storage
import pandas as pd
import matplotlib.pyplot as plt
import string

!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('all')
from nltk.tokenize import word_tokenize

# cleaning up from stopwords
from nltk.corpus import stopwords

# lemmatizing the words
from nltk.stem import WordNetLemmatizer

# vectorizing the tokens
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to /Users/dima/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /Users/dima/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/dima/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /Users/dima/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /Users/dima/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /Users/dima/nltk_data...


In [2]:
# #bucket name
bucket_name = 'flavor-forecast-raw-data'

# # initialize a GCS client
storage_client = storage.Client()

# # get the right bucket
bucket = storage_client.bucket(bucket_name)

In [3]:
# storage_filename = "yelp-dataset/yelp_academic_dataset_business.json"
# local_filename = "yelp_business_raw.json"

# blob = bucket.blob(storage_filename)
# blob.download_to_filename(local_filename)


# # # creating a json file path to take the input form
# # json_file_path = "yelp_business_raw.json"

# # #reading the json file
# # df_business_raw = pd.read_json(json_file_path, lines=True)

# # # creating a destination path
# # csv_file_path = "yelp_business_raw.csv"

# # #converting the output into a csv file
# # df_business_raw.to_csv(csv_file_path, index=False)

In [4]:
# # using storage filename for reviews raw dataset
# storage_filename_review = "yelp-dataset/yelp_academic_dataset_review.json"
# local_filename_review = "yelp_dataset_review_raw.json"

# # downloading the dataset
# blob = bucket.blob(storage_filename_review)
# blob.download_to_filename(local_filename_review)

In [6]:
# creating a json file path to take the input form
json_file_path_review = "/Users/dima/code/Dimasaur/scorecast/notebooks/yelp_academic_dataset_review.json"

#reading the json file
df_review_raw = pd.read_json(json_file_path_review, lines=True)

# creating a destination path
csv_file_path_review = "yelp_review_raw.csv"

#converting the output into a csv file
df_review_raw.to_csv(csv_file_path_review, index=False)

In [7]:
df_review_raw.shape

(6990280, 9)

In [8]:
# load restaurants filter file
res_ids = pd.read_csv("/Users/dima/code/Dimasaur/scorecast/data/restaurants.csv")

In [9]:
all_ids = df_review_raw.business_id

print(f'Number of total unique business IDs in the entire dataset: {all_ids.nunique()}')
print(f'Number of total unique restaurant IDs: {res_ids.business_id.nunique()}')


Number of total unique business IDs in the entire dataset: 150346
Number of total unique restaurant IDs: 52268


In [10]:
df_review_none_filt = df_review_raw[df_review_raw['business_id'].isin(all_ids)]
df_review_res_filt = df_review_raw[df_review_raw['business_id'].isin(res_ids.business_id)]

print(f'Shape of the dataset by including all businesses: {df_review_none_filt.shape}')
print(f'Shape of the dataset by applying restaurant-only filter: {df_review_res_filt.shape}')


Shape of the dataset by including all businesses: (6990280, 9)
Shape of the dataset by applying restaurant-only filter: (4724471, 9)


In [11]:
df_review_res_filt.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15
5,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1,1,2,1,I am a long term frequent customer of this est...,2015-09-23 23:10:31


In [12]:
df_review_fil_pos = df_review_res_filt[df_review_res_filt.stars >= 4]
df_review_fil_neg = df_review_res_filt[df_review_res_filt.stars < 4]

print('Positive reviews are those with 4 and 5 star rating. Negative reviews are considered reviews below stars')
print(f'Number of positive reviews: {df_review_fil_pos.shape[0]}')
print(f'Number of negative reviews: {df_review_fil_neg.shape[0]}')

Positive reviews are those with 4 and 5 star rating. Negative reviews are considered reviews below stars
Number of positive reviews: 3209692
Number of negative reviews: 1514779


In [13]:
reviews = df_review_fil_pos.text
reviews.head()

3     Wow!  Yummy, different,  delicious.   Our favo...
4     Cute interior and owner (?) gave us tour of up...
7     Amazingly amazing wings and homemade bleu chee...
11    Locals recommended Milktooth, and it's an amaz...
12    Love going here for happy hour or dinner!  Gre...
Name: text, dtype: object

In [14]:
def strip_lower(reviews):
    # converting the reviews column into a list before stripping
    reviews_list = list(reviews)
    reviews_list_strip_lower = [review.strip().lower() for review in reviews_list]

    # lowering the letters in the reviews
    for review in reviews_list_strip_lower:
        for punctuation in string.punctuation:
            review = review.replace(punctuation,'')

    return reviews_list_strip_lower

reviews_list_strip_lower = strip_lower(reviews)

In [15]:
df_review_pos = pd.DataFrame(reviews_list_strip_lower,columns=['reviews'])
print(f'DF # of reviews check after strip-lower-punctuation preproc: {df_review_pos.shape[0]==df_review_fil_pos.shape[0]}')

DF # of reviews check after strip-lower-punctuation preproc: True


In [16]:
# sampling the
df_pos_sample = df_review_pos.sample(1000000)
df_pos_sample.head()

,reviews
389677,"best spot for comfort food, especially the roa..."
1779623,we have visited this place twice. the first ti...
2912984,delicious food. the chargrilled oysters were a...
1414067,my family moved to the area last week and take...
2379591,this is definitely the place that rocked us ha...


In [17]:
df_pos_sample['tokenizer'] = df_pos_sample["reviews"].apply(word_tokenize)

In [18]:
def clean_stopwords(df_pos_sample):
    stop_words = set(stopwords.words('english'))
    df_sample_tokens = df_pos_sample.tokenizer

    df_tokens_clean = []
    for list in df_sample_tokens:
        for w in list:
            tokens_clean = [w for w in list if not w in stop_words]
        df_tokens_clean.append(tokens_clean)

    return df_tokens_clean

df_pos_sample['tokenizer_clean'] = clean_stopwords(df_pos_sample)

In [19]:
df_pos_sample.head()

,reviews,tokenizer,tokenizer_clean
389677,"best spot for comfort food, especially the roa...","[best, spot, for, comfort, food, ,, especially...","[best, spot, comfort, food, ,, especially, roa..."
1779623,we have visited this place twice. the first ti...,"[we, have, visited, this, place, twice, ., the...","[visited, place, twice, ., first, time, blue, ..."
2912984,delicious food. the chargrilled oysters were a...,"[delicious, food, ., the, chargrilled, oysters...","[delicious, food, ., chargrilled, oysters, ama..."
1414067,my family moved to the area last week and take...,"[my, family, moved, to, the, area, last, week,...","[family, moved, area, last, week, takeout, fir..."
2379591,this is definitely the place that rocked us ha...,"[this, is, definitely, the, place, that, rocke...","[definitely, place, rocked, us, hard, !, alway..."


In [20]:
def lemmatize_verb_nount(lemmatize_input):
    # lemmatize the verb
    sample_lemmatize_v = []
    for review in lemmatize_input:
        review_lemmatized_verb = [
        WordNetLemmatizer().lemmatize(word,pos='v')
        for word in review
        ]
        sample_lemmatize_v.append(review_lemmatized_verb)

    #lemmatizing the nouns
    sample_lemmatize_v_n = []
    for review in sample_lemmatize_v:
        review_lemmatized_noun = [
        WordNetLemmatizer().lemmatize(word,pos='v')
        for word in review
        ]
        sample_lemmatize_v_n.append(review_lemmatized_noun)

    return sample_lemmatize_v_n

In [21]:
df_pos_sample['tokens_lemm'] = lemmatize_verb_nount(df_pos_sample.tokenizer_clean)

In [22]:
df_pos_sample.head()

,reviews,tokenizer,tokenizer_clean,tokens_lemm
389677,"best spot for comfort food, especially the roa...","[best, spot, for, comfort, food, ,, especially...","[best, spot, comfort, food, ,, especially, roa...","[best, spot, comfort, food, ,, especially, roa..."
1779623,we have visited this place twice. the first ti...,"[we, have, visited, this, place, twice, ., the...","[visited, place, twice, ., first, time, blue, ...","[visit, place, twice, ., first, time, blue, mo..."
2912984,delicious food. the chargrilled oysters were a...,"[delicious, food, ., the, chargrilled, oysters...","[delicious, food, ., chargrilled, oysters, ama...","[delicious, food, ., chargrilled, oyster, amaz..."
1414067,my family moved to the area last week and take...,"[my, family, moved, to, the, area, last, week,...","[family, moved, area, last, week, takeout, fir...","[family, move, area, last, week, takeout, firs..."
2379591,this is definitely the place that rocked us ha...,"[this, is, definitely, the, place, that, rocke...","[definitely, place, rocked, us, hard, !, alway...","[definitely, place, rock, us, hard, !, always,..."


In [23]:
def stringize_tokens(df_pos_sample):
    vector_input = []
    for review in df_pos_sample:
        review_string = " ".join(review)
        vector_input.append(review_string)

    return vector_input

In [24]:
reviews_preproc = stringize_tokens(df_pos_sample.tokens_lemm)
len(reviews_preproc)

1000000

In [25]:
tf_idf_vectorizer = TfidfVectorizer(
                    ngram_range=(2,4),
                    min_df=0.001,
                    max_df=0.1
)

weighted_words = pd.DataFrame(tf_idf_vectorizer.fit_transform(reviews_preproc).toarray(),
                              columns=tf_idf_vectorizer.get_feature_names_out())

In [26]:
sum_tfidf = weighted_words.sum(axis=0)
sum_tfidf_sorted = sum_tfidf.sort_values(ascending=False)

In [ ]:
sum_tfidf_sorted.head(10)

highly recommend    11426.526955
come back           10253.254904
go back             10211.289282
great food          10160.195870
great service        8624.867037
really good          8590.648560
food great           8270.153498
first time           8148.088769
love place           7893.265776
great place          7561.476512
food good            7378.762742
ve ever              7060.009917
good food            7024.425853
service great        6923.187662
new orleans          6002.620622
happy hour           5988.280950
one best             5895.428366
staff friendly       5819.844284
next time            5812.932111
definitely back      5672.290852
make sure            5348.092735
food delicious       5326.410553
food service         5321.610791
ca wait              5171.003330
pretty good          5127.369892
mac cheese           5052.389866
friendly staff       5022.213375
ice cream            4940.949232
every time           4887.581767
customer service     4825.523675
dtype: flo

### Most common phrases in the negative reviews

In [28]:
reviews_neg = df_review_fil_neg.text
reviews_neg.head()

0     If you decide to eat here, just be aware it is...
2     Family diner. Had the buffet. Eclectic assortm...
5     I am a long term frequent customer of this est...
9     Had a party of 6 here for hibachi. Our waitres...
21    Upland is a brewery based out of Bloomington, ...
Name: text, dtype: object

In [29]:
def strip_lower(reviews):
    # converting the reviews column into a list before stripping
    reviews_list = list(reviews)
    reviews_list_strip_lower = [review.strip().lower() for review in reviews_list]

    # lowering the letters in the reviews
    for review in reviews_list_strip_lower:
        for punctuation in string.punctuation:
            review = review.replace(punctuation,'')

    return reviews_list_strip_lower

reviews_list_strip_lower_neg = strip_lower(reviews_neg)

In [30]:
df_review_neg = pd.DataFrame(reviews_list_strip_lower_neg,columns=['reviews'])
print(f'DF # of reviews check after strip-lower-punctuation preproc: {df_review_neg.shape[0]==df_review_fil_neg.shape[0]}')

DF # of reviews check after strip-lower-punctuation preproc: True


In [31]:
# sampling the
df_neg_sample = df_review_neg.sample(500000)
df_neg_sample.head()

,reviews
340427,pass this up. waffle had taste and texture of...
258556,i always support black owned businesses and i'...
846221,"two stars seems a little negative, but...meh, ..."
1401209,thoroughly disappointed in the recent menu rev...
453940,where do i begin with this horrendous experien...


In [32]:
df_neg_sample['tokenizer'] = df_neg_sample["reviews"].apply(word_tokenize)

In [33]:
def clean_stopwords(df_neg_sample):
    stop_words = set(stopwords.words('english'))
    df_sample_tokens = df_neg_sample.tokenizer

    df_tokens_clean = []
    for list in df_sample_tokens:
        for w in list:
            tokens_clean = [w for w in list if not w in stop_words]
        df_tokens_clean.append(tokens_clean)

    return df_tokens_clean

df_neg_sample['tokenizer_clean'] = clean_stopwords(df_neg_sample)

In [34]:
df_neg_sample.head()

,reviews,tokenizer,tokenizer_clean
340427,pass this up. waffle had taste and texture of...,"[pass, this, up, ., waffle, had, taste, and, t...","[pass, ., waffle, taste, texture, rehydrated, ..."
258556,i always support black owned businesses and i'...,"[i, always, support, black, owned, businesses,...","[always, support, black, owned, businesses, 'm..."
846221,"two stars seems a little negative, but...meh, ...","[two, stars, seems, a, little, negative, ,, bu...","[two, stars, seems, little, negative, ,, ..., ..."
1401209,thoroughly disappointed in the recent menu rev...,"[thoroughly, disappointed, in, the, recent, me...","[thoroughly, disappointed, recent, menu, revam..."
453940,where do i begin with this horrendous experien...,"[where, do, i, begin, with, this, horrendous, ...","[begin, horrendous, experience, ?, walked, soo..."


In [35]:
def lemmatize_verb_nount(lemmatize_input):
    # lemmatize the verb
    sample_lemmatize_v = []
    for review in lemmatize_input:
        review_lemmatized_verb = [
        WordNetLemmatizer().lemmatize(word,pos='v')
        for word in review
        ]
        sample_lemmatize_v.append(review_lemmatized_verb)

    #lemmatizing the nouns
    sample_lemmatize_v_n = []
    for review in sample_lemmatize_v:
        review_lemmatized_noun = [
        WordNetLemmatizer().lemmatize(word,pos='v')
        for word in review
        ]
        sample_lemmatize_v_n.append(review_lemmatized_noun)

    return sample_lemmatize_v_n

In [36]:
df_neg_sample['tokens_lemm'] = lemmatize_verb_nount(df_neg_sample.tokenizer_clean)

In [37]:
df_neg_sample.head()

,reviews,tokenizer,tokenizer_clean,tokens_lemm
340427,pass this up. waffle had taste and texture of...,"[pass, this, up, ., waffle, had, taste, and, t...","[pass, ., waffle, taste, texture, rehydrated, ...","[pass, ., waffle, taste, texture, rehydrated, ..."
258556,i always support black owned businesses and i'...,"[i, always, support, black, owned, businesses,...","[always, support, black, owned, businesses, 'm...","[always, support, black, own, businesses, 'm, ..."
846221,"two stars seems a little negative, but...meh, ...","[two, stars, seems, a, little, negative, ,, bu...","[two, stars, seems, little, negative, ,, ..., ...","[two, star, seem, little, negative, ,, ..., me..."
1401209,thoroughly disappointed in the recent menu rev...,"[thoroughly, disappointed, in, the, recent, me...","[thoroughly, disappointed, recent, menu, revam...","[thoroughly, disappoint, recent, menu, revamp,..."
453940,where do i begin with this horrendous experien...,"[where, do, i, begin, with, this, horrendous, ...","[begin, horrendous, experience, ?, walked, soo...","[begin, horrendous, experience, ?, walk, soon,..."


In [38]:
def stringize_tokens(df_neg_sample):
    vector_input = []
    for review in df_neg_sample:
        review_string = " ".join(review)
        vector_input.append(review_string)

    return vector_input

In [39]:
reviews_preproc_neg = stringize_tokens(df_neg_sample.tokens_lemm)
len(reviews_preproc_neg)

500000

In [43]:
tf_idf_vectorizer = TfidfVectorizer(
                    ngram_range=(2,4),
                    min_df=0.01,
                    max_df=0.1
)

weighted_words_neg = pd.DataFrame(tf_idf_vectorizer.fit_transform(reviews_preproc_neg).toarray(),
                              columns=tf_idf_vectorizer.get_feature_names_out())

In [44]:
sum_tfidf_neg = weighted_words_neg.sum(axis=0)
sum_tfidf_sorted_neg = sum_tfidf_neg.sort_values(ascending=False)

In [45]:
sum_tfidf_sorted_neg.head(30)

go back             19830.608432
come back           16469.608666
taste like          13967.838355
food good           13941.844757
customer service    11953.075661
pretty good         11308.742821
first time          10872.477135
look like            9715.354261
take order           7900.374379
get food             7683.351882
really good          7221.921268
seem like            7028.371332
good food            7014.262103
even though          6942.388954
ve ever              6786.930922
20 minutes           6550.077691
much better          6542.896604
next time            6437.487070
service good         6433.534991
10 minutes           6408.383964
nothing special      6333.175198
last time            6295.568919
tell us              6281.478749
15 minutes           6207.687788
would recommend      6194.103457
feel like            6124.454192
food ok              5875.399325
good service         5621.908876
food come            5587.419767
would go             5584.007764
dtype: flo